````

```markdown
# One Batch Backtest Example
This notebook demonstrates running your backtest logic for a single batch of tickers.
```


In [15]:
from src.wrds_pull import fetch_taq_data
from src.strategy import OBIVWAPStrategy
from src.plot import plot_account_balance
from src.performance import evaluate_strategy_performance
import random
import polars as pl
import gc

EX_FILTER = "'Z'"
# EX_FILTER = exchanges = "'Q', 'N', 'T', 'P', 'Z', 'B', 'C', 'X', 'Y', 'A', 'M', 'D', 'I', 'J', 'K', 'L', 'V', 'W', 'U', 'S', 'F', 'G', 'H', 'O', 'R'"
QU_COND_FILTER = "'R'"
START_DATE = '2023-05-10'
END_DATE = '2023-05-11'
START_TIME = (9, 55)
END_TIME = (15, 36)
VWAP_WINDOW = 500
OBI_THRESHOLD = 0
SIZE_THRESHOLD = 0
VWAP_THRESHOLD = 0

In [ ]:
# Choose a batch of tickers (example: first 2 from your filtered list)
with open("data/positive_return_tickers_v1.txt") as f:
    all_filtered = [line.strip() for line in f if line.strip()]

batch = random.sample(all_filtered, 5)
# batch = all_filtered[0:5] 
print("Batch tickers:", batch)

Batch tickers: ['ZION', 'LNC', 'FRT', 'GL', 'SYF']


In [17]:
# Fetch TAQ data for this batch
df = fetch_taq_data(
    tickers=batch,
    exchanges=EX_FILTER,
    quote_conds=QU_COND_FILTER,
    start_date=START_DATE,
    end_date=END_DATE,
    wrds_username='changjulian17'
)
df

Fetching TAQ data from WRDS...
Loading library list...
Done
Data fetched successfully. Query time: 104.04 seconds.


date,time_m,time_m_nano,ex,sym_root,sym_suffix,bid,bidsiz,ask,asksiz,qu_cond,qu_seqnum,natbbo_ind,qu_cancel,qu_source,rpi,ssr,luld_bbo_indicator,finra_bbo_ind,finra_adf_mpid,finra_adf_time,finra_adf_time_nano,finra_adf_mpq_ind,finra_adf_mquo_ind,sip_message_id,natl_bbo_luld,part_time,part_time_nano,secstat_ind,Timestamp
str,str,i64,str,str,str,f64,i64,f64,i64,str,i64,str,str,str,str,str,str,str,str,str,i64,str,str,str,str,str,i64,str,datetime[ns]
"""2023-05-10""","""09:30:00.005927""",36,"""Z""","""ZION""",null,24.85,5,25.39,6,"""R""",4082573,"""2""",null,"""N""",null,"""0""",null,null,null,null,0,null,null,null,null,"""09:30:00.005739""",0,null,2023-05-10 09:30:00.005927360
"""2023-05-10""","""09:30:00.006025""",617,"""Z""","""ZION""",null,24.85,5,25.39,7,"""R""",4082580,"""2""",null,"""N""",null,"""0""",null,null,null,null,0,null,null,null,null,"""09:30:00.005839""",0,null,2023-05-10 09:30:00.006025617
"""2023-05-10""","""09:30:00.014506""",224,"""Z""","""ZION""",null,24.85,5,25.39,8,"""R""",4083130,"""0""",null,"""N""",null,"""0""",null,null,null,null,0,null,null,null,null,"""09:30:00.014324""",0,null,2023-05-10 09:30:00.014506224
"""2023-05-10""","""09:30:00.025596""",499,"""Z""","""ZION""",null,24.85,5,25.39,7,"""R""",4084178,"""0""",null,"""N""",null,"""0""",null,null,null,null,0,null,null,null,null,"""09:30:00.025411""",0,null,2023-05-10 09:30:00.025596499
"""2023-05-10""","""09:30:00.030080""",469,"""Z""","""ZION""",null,24.85,5,25.39,2,"""R""",4084586,"""0""",null,"""N""",null,"""0""",null,null,null,null,0,null,null,null,null,"""09:30:00.029894""",0,null,2023-05-10 09:30:00.030080469
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2023-05-11""","""15:59:59.906925""",919,"""Z""","""ZION""",null,22.65,2,22.75,8,"""R""",88795465,"""0""",null,"""N""",null,"""0""",null,null,null,null,0,null,null,null,"""A""","""15:59:59.906733""",0,null,2023-05-11 15:59:59.906925919
"""2023-05-11""","""15:59:59.906957""",443,"""Z""","""ZION""",null,22.65,2,22.74,2,"""R""",88795466,"""0""",null,"""N""",null,"""0""",null,null,null,null,0,null,null,null,"""A""","""15:59:59.906772""",0,null,2023-05-11 15:59:59.906957443
"""2023-05-11""","""15:59:59.928331""",264,"""Z""","""SYF""",null,27.4,8,27.41,1,"""R""",49618304,"""A""",null,"""C""",null,null,null,null,null,null,0,null,null,null,"""A""","""15:59:59.928101""",0,null,2023-05-11 15:59:59.928331264


In [18]:
positive_return_tickers = []

stock_tickers = df["sym_root"].unique().to_list()
print("Tickers in data:", stock_tickers)

for ticker in stock_tickers:
    print(f"Processing {ticker}...")
    ticker_data = df.filter(pl.col("sym_root") == ticker)
    strategy = OBIVWAPStrategy(
        vwap_window=VWAP_WINDOW, 
        obi_threshold=OBI_THRESHOLD, 
        size_threshold=SIZE_THRESHOLD,
        vwap_threshold=VWAP_THRESHOLD,
        start_time=START_TIME, 
        end_time=END_TIME
    )
    ticker_data = strategy.generate_signals(ticker_data)
    backtest_data = strategy.backtest(ticker_data)
    # plot_account_balance(backtest_data)  # Uncomment to plot
    metrics = evaluate_strategy_performance(backtest_data)
    print(metrics)
    if metrics.get("Total_Returns", 0) > 0:
        positive_return_tickers.append(ticker)
    del ticker_data, backtest_data
    gc.collect()

print("Positive return tickers:", positive_return_tickers)

Tickers in data: ['FRT', 'ZION', 'SYF', 'LNC', 'GL']
Processing FRT...

PERFORMANCE FRT STATISTICS:
Total returns: 741.44%
Max drawdown: -1.58%

DAILY SHARPE RATIOS:
shape: (2, 2)
┌─────────────────────┬────────────────────┐
│ Timestamp           ┆ Daily_Sharpe_Ratio │
│ ---                 ┆ ---                │
│ datetime[ns]        ┆ f64                │
╞═════════════════════╪════════════════════╡
│ 2023-05-10 00:00:00 ┆ 1.246245           │
│ 2023-05-11 00:00:00 ┆ 1.097961           │
└─────────────────────┴────────────────────┘
{'Total_Returns': np.float64(741.438), 'Max_Drawdown': np.float64(-1.577787755698723), 'Daily_Sharpe_Ratios': shape: (2, 2)
┌─────────────────────┬────────────────────┐
│ Timestamp           ┆ Daily_Sharpe_Ratio │
│ ---                 ┆ ---                │
│ datetime[ns]        ┆ f64                │
╞═════════════════════╪════════════════════╡
│ 2023-05-10 00:00:00 ┆ 1.246245           │
│ 2023-05-11 00:00:00 ┆ 1.097961           │
└────────────────────